#### Imports

In [29]:
import pandas as pd
import gzip
import datetime
import requests
import csv
# import contractions
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import d2l
# from textblob import TextBlob
from nltk.corpus import stopwords
from mxnet import init
from mxnet.gluon import loss as gloss
from mxnet import gluon
from pandas.io.json import json_normalize

## Load Data

Data stored from various categories. Categories include (

Method to load data taken from : 

In [19]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

## Clean Data

## Useful Code Snipits

##### Group by asin which is tthe product ID

In [28]:
# print("test1")
automotive = getDF('category_data/reviews_Automotive_5.json.gz')
baby = getDF('category_data/reviews_Baby_5.json.gz')
beauty = getDF('category_data/reviews_Beauty_5.json.gz')
grocery = getDF('category_data/reviews_Grocery_and_Gourmet_Food_5.json.gz')
musical = getDF('category_data/reviews_Musical_Instruments_5.json.gz')
office = getDF('category_data/reviews_Office_Products_5.json.gz')
patio = getDF('category_data/reviews_Patio_Lawn_and_Garden_5.json.gz')
pet = getDF('category_data/reviews_Pet_Supplies_5.json.gz')
books = getDF("category_data/reviews_Books_5.json.gz")
reviews = getDF("category_data/reviews_CDs_and_Vinyl_5.json.gz")
cell = getDF("category_data/reviews_Cell_Phones_and_Accessories_5.json.gz")
clothing = getDF("category_data/reviews_Clothing_Shoes_and_Jewelry_5.json.gz")
electronics = getDF("category_data/reviews_Electronics_5.json.gz")
health = getDF("category_data/reviews_Health_and_Personal_Care_5.json.gz")
home = getDF("category_data/reviews_Home_and_Kitchen_5.json.gz")
sports = getDF("category_data/reviews_Sports_and_Outdoors_5.json.gz")
tools = getDF("category_data/reviews_Tools_and_Home_Improvement_5.json.gz")
toys = getDF("category_data/reviews_Toys_and_Games_5.json.gz")
games = getDF("category_data/reviews_Video_Games_5.json.gz")

# print("test2")
df = pd.concat([automotive, baby, beauty, grocery, musical, office, patio, pet, books, reviews, cell, clothing, electronics, health, home, sports, tools, toys, games])
df["reviewTime"] = pd.to_datetime(df["reviewTime"])

mask = (df["reviewTime"] >= '2010-01-01') & (df["reviewTime"] <= '2014-07-31')
df = df.loc[mask]
# print("test3")
df.to_csv("category_data/filtered_joined_dataset")
# print("test4")

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5.0,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5.0,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2.0,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5.0,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5.0,Still the best,1375574400,"08 4, 2013"
5,A2JFOHC9W629IE,B00000JBLH,scott_from_dallas,"[10, 12]","While I don't have an MBA, it's hard to believ...",5.0,Every MBA student and grad should get one,1011744000,"01 23, 2002"
6,A38NELQT98S4H8,B00000JBLH,W. B. Halper,"[3, 4]",I've had an HP 12C ever since they were first ...,5.0,A workhorse of a calculator,1168992000,"01 17, 2007"
7,AA8M6331NI1EN,B00000JBLH,ZombieMom,"[0, 0]",Bought this for my boss because he lost his. ...,5.0,Fast shipping & great price for this awesome c...,1384387200,"11 14, 2013"
8,A25C2M3QF9G7OQ,B00000JBLU,Comdet,"[3, 3]","This is a well-designed, simple calculator tha...",5.0,"Nice design, works well, great value",1291680000,"12 7, 2010"
9,A1RTVWTWZSIC94,B00000JBLU,"Hb ""Black Beauty""","[0, 0]","I love this calculator, big numbers and calcul...",5.0,Love It!!!!!!!!!,1385942400,"12 2, 2013"


## Feature : The star rating

The review-assigned product star rating

#### EDA

## Feature : Subjectivity 

The probability of a review and its sentences being subjective. https://textblob.readthedocs.io/en/dev/ and https://planspace.org/20150607-textblob_sentiment/

In [24]:
TextBlob("not a very great calculation").sentiment
## Sentiment(polarity=-0.3076923076923077, subjectivity=0.5769230769230769)

Sentiment(polarity=-0.3076923076923077, subjectivity=0.5769230769230769)

#### EDA

## Feature : TF-IDF  |  Product features

Attempt to identify the presence of important topics

In [14]:
def tf_idf(document):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(document)
    return X.toarray()

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

# A Tokenizer that tokenize a sequence of alphabetical character and drop everything else
tokenizer = RegexpTokenizer(r'\w+') 

def get_aspects(x):
    import contractions
    doc = contractions.fix(x)
    doc=tokenizer.tokenize(doc) ## Tokenize and extract grammatical components
    doc=list(map(lambda i: i.lower(),doc)) ## Normalize text to lower case
    doc=[i for i in doc if i not in stop_words] ## Remove common words and retain words not in stop words
    doc = [lemmatizer.lemmatize(j) for j in doc]
    doc=pd.Series(doc)
    doc=doc.value_counts().head(10).index.tolist() ## Get 10 most frequent words
    return doc

get_aspects(df_group.get_group("120401325X")["reviewText"][5])

NameError: name 'df_group' is not defined

#### EDA

## Feature : Sensitivity Analysis

Looking for use of pos/neg words

In [17]:
test_subset = tokenizer.tokenize(df_group.get_group("120401325X")["reviewText"][3])
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

for word in test_subset:
    if (sid.polarity_scores(word)['compound']) >= 0.5:
        pos_word_list.append(word)
    elif (sid.polarity_scores(word)['compound']) <= -0.5:
        neg_word_list.append(word)
    else:
        neu_word_list.append(word)                

print('Positive :',pos_word_list)        
print('Neutral :',neu_word_list)    
print('Negative :',neg_word_list)



NameError: name 'df_group' is not defined

#### EDA

## Feature : Readability 

Measures how easy a text is to read

In [ ]:
def readabilityMetrics(text):
    response = requests.post("https://ipeirotis-readability-metrics.p.rapidapi.com/getReadabilityMetrics",
      headers={
        "X-RapidAPI-Host": "ipeirotis-readability-metrics.p.rapidapi.com",
        "X-RapidAPI-Key": "818e0f0b58mshe4a4d5842f257bep1cc5a6jsne73e02710bab",
        "Content-Type": "application/x-www-form-urlencoded"
      },
      params={
        "text":text                        
      }
    )
    rowEntry = response.json()
    rowEntry["TEXT"] = text
    return json_normalize(rowEntry)

#### EDA

## Feature : Spelling Errors

In [ ]:
def numberOfSpellingErrors(text):
    response = requests.get("https://montanaflynn-spellcheck.p.rapidapi.com/check/",
      headers={
            "X-RapidAPI-Host": "montanaflynn-spellcheck.p.rapidapi.com",
            "X-RapidAPI-Key": "818e0f0b58mshe4a4d5842f257bep1cc5a6jsne73e02710bab"
      },
      params={
        "text":text                        
      }
    )
    return len(response.json()['corrections'])

#### EDA

## Define the Model

Since we are predicting a value between [0.0, 5.0] we will be using a linear regression model. 

***CITE*** Method from D2l 

In [30]:
net.initialize(init.Normal(sigma=0.01))

loss = gloss.L2Loss()
def get_net():
    net = nn.Sequential()
    net.add(nn.Dense(1))
    net.initialize()
    return net


def train(net, train_features, train_labels, test_features, test_labels, num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = gdata.DataLoader(gdata.ArrayDataset(
    train_features, train_labels), batch_size, shuffle=True) # The Adam optimization algorithm is used here.
    trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': learning_rate, 'wd': weight_decay}) 
    for epoch in range(num_epochs):
        for X, y in train_iter: 
            with autograd.record(): 
                l = loss(net(X), y)
            l.backward()
            trainer.step(batch_size)
        train_ls.append(log_rmse(net, train_features, train_labels)) 
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels)) 
    return train_ls, test_ls



## Optimization Algorithm
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

## Define the Training Functions

## Classify the Testing Set

## Measure Accuracy

## Questions at Check-In

1. How can we make our project more novel? 
2.
3.
4.
5.
6.